In [ ]:
#@title نصب
!git clone https://github.com/yaranbarzi/roop
%cd roop

import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q

!pip install onnxruntime-gpu -q

!pip install numpy opencv-python onnx insightface psutil pillow tqdm -q
!pip install customtkinter tkinterdnd2 -q

!pip install tensorflow-cpu -q

!pip install gfpgan realesrgan basicsr -q
!pip install opennsfw2 keras protobuf -q

!pip install ipywidgets -q

import gc
gc.collect()

from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output

print("✅ Setup complete with Face Enhancer support.")

In [ ]:
#@title آپلود عکس
%cd /content/roop

# Clean up previous uploads
if os.path.exists("source.png"):
    os.remove("source.png")

uploaded = files.upload()

if uploaded:
    # Get the filename of the uploaded file
    original_filename = next(iter(uploaded))
    # Rename and save it as source.png
    os.rename(original_filename, "source.png")
    print("✅ Source image uploaded and saved as 'source.png'")
else:
    print("❌ No file uploaded.")

In [ ]:
#@title آپلود ویدیوی هدف
%cd /content/roop

# Clean up previous uploads
if os.path.exists("target.mp4"):
    os.remove("target.mp4")

uploaded = files.upload()

if uploaded:
    # Get the filename of the uploaded file
    original_filename = next(iter(uploaded))
    # Rename and save it as target.mp4
    os.rename(original_filename, "target.mp4")
    print("✅ Target video uploaded and saved as 'target.mp4'")
else:
    print("❌ No file uploaded.")

In [ ]:
#@title اجرای دیپ فیک
enable_enhancer = True #@param {type:"boolean"}
face_selection_mode = "Swap Default Face (Fastest)" #@param ["Swap Default Face (Fastest)", "Swap ALL faces in video", "Swap a SPECIFIC face"]
specific_face_number = 0 #@param {type:"number"}
execution_provider = "CPU (Stable)" #@param ["CPU (Stable)", "GPU (May crash with CuDNN error)"]

import os
from google.colab import files
from IPython.display import display, HTML
import gc

%cd /content/roop
gc.collect()

# تنظیم execution provider
if "CPU" in execution_provider:
    provider = "cpu"
    # غیرفعال کردن GPU برای جلوگیری از خطای CuDNN
    os.environ['CUDA_VISIBLE_DEVICES'] = ''
else:
    provider = "cuda"
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# ساخت کامند
command = ("python run.py -s 'source.png' -t 'target.mp4' -o 'output.mp4' "
           f"--execution-provider {provider} "
           "--keep-fps --output-video-quality 1 --max-memory 8 --output-video-encoder libx264 ")

# تنظیم frame processors
if enable_enhancer:
    command += "--frame-processor face_swapper face_enhancer "
else:
    command += "--frame-processor face_swapper "

# تنظیمات face selection
if "ALL faces" in face_selection_mode:
    command += "--many-faces "
elif "SPECIFIC face" in face_selection_mode:
    command += f"--reference-face-position {specific_face_number} "

print("""Final optimized command to be executed:""")
print(f"""{command}""")
print(f"""--- Starting process with {execution_provider}... ---""")

!{command}

gc.collect()
print("""--- Process finished! ---""")

if os.path.exists("output.mp4"):
    print("""✅ Success! Downloading the final video...""")
    files.download("output.mp4")
else:
    print("""❌ Output file 'output.mp4' not found. Check the logs above for errors.""")
    print("Available files:")
    !ls -la *.mp4 *.png